In [21]:
import os
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


In [72]:
import tensorflow as tf
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs['accuracy'] > 0.84 and logs['val_accuracy'] > 0.6:
            print("\nReached desired accuracy, stopping training.")
            self.model.stop_training = True

In [53]:
# target_size = (128, 128)
batch_size = 32
num_classes = 31  # Adjust based on your number of classes

In [54]:
data_directory = 'datasets_character_anime'
image_filenames = [os.path.join(data_directory, folder, filename) for folder in os.listdir(data_directory) if os.path.isdir(os.path.join(data_directory, folder)) for filename in os.listdir(os.path.join(data_directory, folder)) if filename.endswith(('.jpg', '.png'))]


In [55]:
for filename in image_filenames:
    img = Image.open(filename)
    img = img.resize((128, 128))
    img.save(filename)

In [63]:
datagen = ImageDataGenerator(
        rescale=1./255,  # normalize pixel values to be between 0 and 1
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest',
        validation_split=0.2
    )

train_generator = datagen.flow_from_directory(
    data_directory,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    data_directory,
    target_size=(128, 128),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 248 images belonging to 31 classes.


Found 60 images belonging to 31 classes.


In [64]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(3, 3), activation='relu', input_shape=(128, 128, 3)))  # Input shape is (64, 64, 1) for grayscale images
model.add(MaxPooling2D())
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=31, activation='softmax'))

In [58]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(128, 128, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=num_classes, activation='softmax'))

In [65]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [73]:

num_epochs = 100
custom_callback = CustomCallback()
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[custom_callback],
    epochs=num_epochs
)

Epoch 1/100
7/7 [==============================] - 1s 156ms/step - loss: 0.1085 - accuracy: 0.9537 - val_loss: 3.3111 - val_accuracy: 0.4375
Epoch 2/100
7/7 [==============================] - 1s 135ms/step - loss: 0.1831 - accuracy: 0.9491 - val_loss: 4.4179 - val_accuracy: 0.3750
Epoch 3/100
7/7 [==============================] - 1s 138ms/step - loss: 0.1712 - accuracy: 0.9375 - val_loss: 3.5491 - val_accuracy: 0.4062
Epoch 4/100
7/7 [==============================] - 1s 130ms/step - loss: 0.1685 - accuracy: 0.9398 - val_loss: 4.5766 - val_accuracy: 0.4062
Epoch 5/100
7/7 [==============================] - ETA: 0s - loss: 0.1158 - accuracy: 0.9630
Reached desired accuracy, stopping training.
7/7 [==============================] - 1s 130ms/step - loss: 0.1158 - accuracy: 0.9630 - val_loss: 3.1010 - val_accuracy: 0.6562


In [74]:
model.save('test.h5')

c:\Users\ACER\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
